### Método de Taylor 
El método de Taylor requiere los siguientes pasos para su solución:
- Tener la ecuación de forma: $ \frac{dy}{dt} = f $, de esta tenemos que sacar tantas derivadas implicitas como el orden menos uno, por ejemplo si nos dice de orden 4 vamos a sacar 3 derivadas implicitas y en cada una se sustituye *y'* por *f*. Hasta el momento todos los ejercicios son de orden 4.
- El ejercicio no proporciona un *h* y un intervalo, o nos puede dar los nodos. Simpre con dos de los datos podemos sacar el otro.
- Una vez tenemos las derivadas implicitas, *h* y los nodos, aplicamos la fórmula de Taylor de orden superior (Abajo aparece la fórmula en Python ya lista)

Ejemplo 1 de clase 14 y video: https://youtu.be/wH5H9m_XfC0?si=qH5u4paBWGz_9WVz

Emplee el método de Taylor de orden 4 pra obtener la solución aproximada a la siguiente ecuación diferencial (En realidad sería al siguiente problema de valores iniciales):

$$ e^{2t}dy = dt - 2ye^{2t}dt, \ \ \ 0 \leq t \leq 0.5, \ \ \ y(0)= 1/10, \ \ \ h= 1/8 $$

Solución:

La función despejada en la forma $ \frac{dy}{dt} = f $  y las 3 primeras derivadas de y', luego de sustituir esta en ellas y simplicar cada una nos da:

$$ F'(t) = f(t, y) = y' = e^{-2t} - 2y $$
$$ F''(t) = f'(t, y) = -4e^{-2t} + 4y $$
$$ F'''(t) = f''(t, y) = 12e^{-2t} - 8y $$
$$ F^{(4)} (t) = f'''(t, y) = -32e^{-2t} + 16y $$

Con estos datos pasamos a las solución en Python:


In [36]:
from sympy import *


fy = symbols("fy", cls=Function)
t, y = symbols("t y")

h= 0.125

# Declaro mi ecuación diferencial y mi función ya que necesito ambas para calcular
# La solución exacta y las aproximaciones, no encontré otra forma de hacerlo en Python
# En la ecuación las "y" siempre seran fy(t)
eq = Eq(fy(t).diff(), exp(-2*t) - 2*fy(t))
# La segunda ecuación se escribe de forma simbólica normal de Sympy
f = exp(-2*t)-2*y

# Calculamos la solucion particular F, si nos da una expresión
# con números complejos, es que esta EDI no tiene solución simbólica en los reales
condiciones_iniciales = {fy(0):0.1}
F = dsolve(eq, ics=condiciones_iniciales)
F

Eq(fy(t), (t + 0.1)*exp(-2*t))

In [37]:
from numpy import arange

# para que arrange funcione adecuadamente, al segundo argumento (stop) hay que
# agregarle la mitad de h para que tome en cuenta el número en stop
# Ojo: Si lo decimales no son adecuados, hay que redondearlos
T = [t for t in arange(start=0, stop=0.5 + h/2, step=h)]
T

[0.0, 0.125, 0.25, 0.375, 0.5]

In [38]:
# Asigno variables a las derivadas
d1f =  -4*exp(-2*t) + 4*y
d2f = 12*exp(-2*t) - 8*y
d3f = -32*exp(-2*t) + 16*y

# Guardo mis resultados aproximados en una lista Y, inicio en el
# valor que me da la condición inicial, en este caso 1/10
Y = [0.1]

# Este bucle for calcula lo Y, para uno de orden menor solo abria que quitar las
# líneas correspondientes, para uno de grado mayor agregarlo.
for i in range(len(T)-1):
    Y.append(
        Y[i] + 
        h*f.subs({t: T[i],y: Y[i]}) + 
        ((h**2)/factorial(2)) * d1f.subs({t: T[i],y: Y[i]}) +
        ((h**3)/factorial(3)) * d2f.subs({t: T[i],y: Y[i]}) +
        ((h**4)/factorial(4)) * d3f.subs({t: T[i],y: Y[i]})
    )

Y

[0.1,
 0.175211588541667,
 0.212257538972750,
 0.224342066811973,
 0.220695310585029]

In [39]:
# Si la solución particular es viable,
# calculamos los valores exactos
solucion_exacta = []
for i in range(len(T)):
    solucion_exacta.append(float(F.rhs.subs({t: T[i]})))
solucion_exacta

[0.1,
 0.1752301761910661,
 0.21228573089942168,
 0.22437411255198197,
 0.2207276647028654]

In [40]:
from utils import imprimir_tabla

lista_tabla = [["t_i", "y(t_i) Aprox", "F(t_i)Exacta", "error", ]]

for i in range(len(T)):
    lista_tabla.append(
        [
            str(T[i]),
            str(Y[i]),
            str(solucion_exacta[i]),
            str(abs(Y[i] - solucion_exacta[i])),
        ]
    )

print("A continuación tenemos los valores de cada t")
imprimir_tabla(lista_tabla)

A continuación tenemos los valores de cada t


┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ t_i   ┃ y(t_i) Aprox      ┃ F(t_i)Exacta        ┃ error               ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ 0.0   │ 0.1               │ 0.1                 │ 0.0                 │
│ 0.125 │ 0.175211588541667 │ 0.1752301761910661  │ 1.85876493994375e-5 │
│ 0.25  │ 0.212257538972750 │ 0.21228573089942168 │ 2.81919266718333e-5 │
│ 0.375 │ 0.224342066811973 │ 0.22437411255198197 │ 3.20457400088403e-5 │
│ 0.5   │ 0.220695310585029 │ 0.2207276647028654  │ 3.23541178363917e-5 │
└───────┴───────────────────┴─────────────────────┴─────────────────────┘

In [41]:
# Si nos piden una valor aproximado interpolado que no esta en T
# Utilizamos Neville

# Calculo el valor en y(0.3) con interpolacion de Neville

from metodos_interpolacion import neville

matriz = neville(T, Y, 0.3)

# En esta lista hay que agregar len(T) columnas
lista_para_tabular = [["Qx0", "Qx1", "Qx2", "Qx3", "Qx4"]]
for fila in matriz:
    nueva_fila = []
    for celda in fila:
        nueva_fila.append(str(celda))
    lista_para_tabular.append(nueva_fila)

imprimir_tabla(lista_para_tabular)

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ Qx0                 ┃ Qx1                 ┃ Qx2                 ┃ Qx3                 ┃ Qx4                 ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ 0.1                 │ 0.0                 │ 0.0                 │ 0.0                 │ 0.0                 │
│ 0.17521158854166666 │ 0.2805078125        │ 0.0                 │ 0.0                 │ 0.0                 │
│ 0.21225753897274985 │ 0.2270759191451831  │ 0.2163895404742197  │ 0.0                 │ 0.0                 │
│ 0.22434206681197313 │ 0.21709135010843916 │ 0.22008672081946234 │ 0.21934728475041385 │ 0.0                 │
│ 0.220695310585029   │ 0.22653012054813962 │ 0.21897910419637925 │ 0.21956983306202357 │ 0.21948081373737965 │
└─────────────────────┴─────────────────────┴─────────────────────┴─────────────────────┴─────────────────────┘

In [42]:
# El valor interpolado nos tiene que dar algo entre sus valores vecinos
print("El valor interpolado de y(0.3) es:", matriz[len(T)-1][len(T)-1])

El valor interpolado de y(0.3) es: 0.21948081373737965
